# Use the Zeitbedarfswert calculator prototype with OTAnalytics

> Note: The base path is set in `.vscode/settings.json` to workspace path. Otherwise Jupyter can't import OTAnalytics.

## Import the prototype modules

Before working with events, you need to import the event_processor to import and convert the events to a shaped data frame.

In [ ]:
# Import libraries and modules
# OTAnalytics modules
from OTAnalytics.plugin_prototypes.event_parser.event_parser import EventParser
from OTAnalytics.plugin_prototypes.zeitbedarf_calculator.zeitbedarf_calculator import (
    ZeitbedarfsCalculator,
)

%load_ext autoreload
%autoreload 2

## Set config variables

In the current version, configs for prototypes are stored in dictionaries within the Jupyter notebook. In a future version, the will likely be stored in a config file.

The path to the eventlists can be a path to a single event list or a folder (all otevent files within this folder will be imported).

At each section, the counts will be calculated by direction using the flows. Therefore, you have to name the directions (default: "in" and "out"). "first_to_last_section" names the direction in which the given section is intersected first by the flow. "last_to_first_section" names the direction in which the given section is intersected last.

In [ ]:
# Set env parameters and path
CONFIG = {
    "CITY_NAME": "Leipzig",
    "INTERSECTION_NAME": "Merseburger Straße - Demmeringstraße",
    "TIME_FORMAT": "%d.%m%.%y %H:%M Uhr",
    "FILTER_CLASS": [],
    "FILTER_SECTION": [],
    "EVENTLIST_PATH": "<YOUR PATH>",
    "SECTIONSLIST_PATH": "<YOUR PATH>",
    "FROM_TIME": "2022-11-01 06:00:00",
    "TO_TIME": "2022-11-03 22:00:00",
    "INTERVAL_LENGTH_MIN": 60,  # in minutes
    "SIGNALPROG_PATH": "<YOUR PATH>",
    "SIGNALPROG_MAPPER_PATH": "<YOUR PATH>",
    "SIGNAL_TIME_GAP": 3.0,  # time in seconds that is added on the signal events timestamps
    "ZB_CUTOFF": 5,  # maximum time in seconds from signal changing to red-yellow to sction-enter event to be considered as Zeitbedarfswert
    "FF_TIME_ENTER_SCENE_TO_SECTION": {  # time in seconds to drive from scene entry to the section in free flow conditions
        "Haltelinie_S8-9": 3,
        "Haltelinie_S4-5-6": 3,
        "Haltelinie_S1-2-3": 2,
    },
    "DIRECTION_NAMES": {
        "first_to_last_section": "in",
        "last_to_first_section": "out",
    },
}

## Import Events

First, we need to import the event file(s) and process it to a eventlist.

In [ ]:
event_processor = EventParser(CONFIG)
events = event_processor.process_events()

## Calculate Zeitbedarfe

With the prototype of the zeitbedarfscalculator, using signal protocols or events given in the config, the zeitbedarfswerte for the given sections is calculated.

For each Haltelinie, a section should be defined and be laying over the geometry of the real Haltelinie.

The signal protocols have to be stored in the format defined in the `example_signal_program.csv`.

With the `example_signal_program_mapper.json`, the signals are linked to the sections. 

The Zeitbedarfswerte can be plotted using the `plot_zb` function. 

In [ ]:
zb_calculator = ZeitbedarfsCalculator(CONFIG, events)
events_zb = zb_calculator.calculate_zbw()
zb_calculator.plot_zb(
    # x="queue_position",
    color="time_interval_t",
    # xaxis_title="Uhrzeit",
    category_orders={
        "q": list(events_zb["q"].sort_values().unique()),
        "time_interval_t": list(events_zb["time_interval_t"].sort_values().unique()),
    },
    facet_row="section_id",
)  # , facet_row="section_id" /